# TESTS PER CAPITOLO 4

# Install Essentia and librosa for simple tasks

In [ ]:
!pip install essentia
!pip install librosa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 73.9 MB/s eta 0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.4/291.4 MB 6.0 MB/s eta 0:00:00


# BPM Music feature extraction models/libraries for tempo test

In [ ]:

# unistall tensorflow if needed
# !pip uninstall tensorflow -y

!pip install essentia-tensorflow

In [ ]:
#import json and assign column to list, returns this list:
def list_classes_from_json(file_path, column_name):
  import json
  # Load the classes from the JSON file
  with open(file_path, 'r') as f:
      data = json.load(f)
  # Extract the classes
  list_classes = data[column_name]
  # OUTPUT:
  return list_classes


#process audio for essentia
def processed_audio_essentia(fileaudiopath):
  from essentia.standard import MonoLoader
  import librosa
  # Load your audio file
  y, sr = librosa.load(fileaudiopath)
  # Process the audio
  audio = MonoLoader(filename=fileaudiopath, sampleRate=sr, resampleQuality=4)()
  # OUTPUT:
  return audio

#creates CSV of column1, column2 ES. filename | result
def create_CSV(column1, column2, filename):
  import csv
  with open(filename, 'w', newline='') as csvfile:
      writer = csv.writer(csvfile)
      for item1, item2 in zip(column1, column2):
        writer.writerow([item1, item2])


In [ ]:
#input loop to test with all the files: librosa.beat.beat_track

import os

# INPUT
directory_path = '/content/loops per prove BPM detection'



filename_column = []
BPM_column = []
# Loop through each file in the directory
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)

    # Check if it's a file (not a subdirectory)
    if os.path.isfile(file_path):

        # librosa.beat.beat_track
        # MOLTO IMPRECISO MA ANCHE MOLTO LENTO

        # PROCESS:
        import librosa

        def get_bpm(fileaudiopath):
          # Load an audio file
          y, sr = librosa.load(fileaudiopath)
          tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
          return tempo

        # OUTPUT:
        BPM = get_bpm(file_path)
        filename_column.append(filename)
        BPM_column.append(BPM)
        print(f'{filename} | {BPM}')


create_CSV(filename_column, BPM_column, "test_BPM_librosa.beat.beat_track.csv")

The Winstons - Amen Brother drum loop 138 BPM.wav | [92.28515625]
mi manchi 1 4bars drop, 125bpm.mp3 | [123.046875]
garage solo drum loop 134 bpm V0.mp3 | [135.99917763]
idea edo suno steelpan V1.1.mp3 | [143.5546875]
african drum fills, 128 bpm..wav | [172.265625]
techno kick, 135 bpm..wav | [135.99917763]
prova brano fred again stile turn off the lights 134bpm V4.flp.mp3 | [135.99917763]
bulgarian-choir-streets_D_minor.wav | [184.5703125]
garage V5, 140bpm.mp3 | [69.83741554]
mi manchi 2 V4.7.mp3 | [95.703125]


In [ ]:
# Essentia RhythmExtractor2013
import os

# INPUT
directory_path = '/content/loops per prove BPM detection'


filename_column = []
BPM_column = []
# Loop through each file in the directory
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)

    # Check if it's a file (not a subdirectory)
    if os.path.isfile(file_path):


        # Essentia RhythmExtractor2013
        # TENDENZIALMENTE PRECISO, MA CON ROUND, A VOLTE SBAGLIA

        # PROCESS
        import essentia
        import essentia.standard as es

        loader = es.MonoLoader(filename=file_path)
        audio = loader()

        # Optional: Normalize the audio
        audio = audio / max(abs(audio))

        # Initialize the RhythmExtractor2013 algorithm
        rhythm_extractor = es.RhythmExtractor2013(method="multifeature") #methods: multifeature, degara, beatspectrum

        # Extract rhythm features
        bpm, beats, beats_confidence, bpm_estimates, bpm_confidences = rhythm_extractor(audio)


        filename_column.append(filename)
        BPM_column.append(bpm)

        print(f'{filename} | {bpm}')


create_CSV(filename_column, BPM_column, "test_BPM_Essentia RhythmExtractor2013.csv")

# LIMITATIONS:
# suggest to work with 4/4 tempo
# if the estimated tempo is 134.51, it will become 135, leading to a possible slightly incorrect bpm value.
# -> TO-DO: in a vst implementation, the bpm could be taken from the DAW infos, and not from the listened file.
# sometimes it can guess the tempo as half-tempo (es. 70bpm instead of 140bpm): this will influence the generation...
# -> TO-DO: show the detected bpm in the user interface, and add a button to duplicate or half the bpm

The Winstons - Amen Brother drum loop 138 BPM.wav | 137.83700561523438
mi manchi 1 4bars drop, 125bpm.mp3 | 125.05763244628906
garage solo drum loop 134 bpm V0.mp3 | 132.69305419921875
idea edo suno steelpan V1.1.mp3 | 141.87591552734375
african drum fills, 128 bpm..wav | 127.84871673583984
techno kick, 135 bpm..wav | 135.19444274902344
prova brano fred again stile turn off the lights 134bpm V4.flp.mp3 | 133.70289611816406
bulgarian-choir-streets_D_minor.wav | 94.52335357666016
garage V5, 140bpm.mp3 | 69.9760513305664
mi manchi 2 V4.7.mp3 | 144.5546417236328


In [ ]:
# !pip install essentia-tensorflow #if not yet installed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.4/291.4 MB 4.4 MB/s eta 0:00:00


In [ ]:
# download the TempoCNN models from essentia
!wget https://essentia.upf.edu/models/tempo/tempocnn/deepsquare-k16-3.pb
!wget https://essentia.upf.edu/models/tempo/tempocnn/deeptemp-k4-3.pb
!wget https://essentia.upf.edu/models/tempo/tempocnn/deeptemp-k16-3.pb

--2024-10-31 04:30:02--  https://essentia.upf.edu/models/tempo/tempocnn/deepsquare-k16-3.pb
Resolving essentia.upf.edu (essentia.upf.edu)... 84.89.139.43
Connecting to essentia.upf.edu (essentia.upf.edu)|84.89.139.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4833690 (4.6M) [application/octet-stream]
Saving to: ‘deepsquare-k16-3.pb’

deepsquare-k16-3.pb 100%[===================>]   4.61M   841KB/s    in 6.1s    

2024-10-31 04:30:09 (779 KB/s) - ‘deepsquare-k16-3.pb’ saved [4833690/4833690]

--2024-10-31 04:30:09--  https://essentia.upf.edu/models/tempo/tempocnn/deeptemp-k4-3.pb
Resolving essentia.upf.edu (essentia.upf.edu)... 84.89.139.43
Connecting to essentia.upf.edu (essentia.upf.edu)|84.89.139.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 147561 (144K) [application/octet-stream]
Saving to: ‘deeptemp-k4-3.pb’

deeptemp-k4-3.pb    100%[===================>] 144.10K   327KB/s    in 0.4s    

2024-10-31 04:30:10 (327 KB/s) 

In [ ]:
 # Essentia TempoCNN
import os

# INPUT
directory_path = '/content/loops per prove BPM detection'


print("deepsquare-k16")
filename_column = []
BPM_column = []
# Loop through each file in the directory
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)

    # Check if it's a file (not a subdirectory)
    if os.path.isfile(file_path):


        # Essentia TempoCNN
        # 3 different models: the k16 ones are more precise, using more resources
        # SEMPRE PRECISO, NON SBAGLIA MAI (PER ORA)

        # PROCESS:
        from essentia.standard import MonoLoader, TempoCNN

        import librosa
        y, sr = librosa.load(file_path)

        audio = MonoLoader(filename=file_path, sampleRate=sr, resampleQuality=4)()

        model1 = TempoCNN(graphFilename="/content/deepsquare-k16-3.pb")
        global_tempo, local_tempo, local_tempo_probabilities = model1(audio)
        # print("Estimated BPM:", global_tempo) #134


        filename_column.append(filename)
        BPM_column.append(global_tempo)

        print(f'{filename} | {global_tempo}')

create_CSV(filename_column, BPM_column, "test_BPM_TempoCNN[deepsquare-k16].csv")






print("deeptemp-k4")
filename_column = []
BPM_column = []
# Loop through each file in the directory
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)

    # Check if it's a file (not a subdirectory)
    if os.path.isfile(file_path):


        # Essentia TempoCNN
        # 3 different models: the k16 ones are more precise, using more resources
        # SEMPRE PRECISO, NON SBAGLIA MAI (PER ORA)

        # PROCESS:
        from essentia.standard import MonoLoader, TempoCNN

        import librosa
        y, sr = librosa.load(file_path)

        audio = MonoLoader(filename=file_path, sampleRate=sr, resampleQuality=4)()

        model2 = TempoCNN(graphFilename="/content/deeptemp-k4-3.pb")
        global_tempo, local_tempo, local_tempo_probabilities = model2(audio)
        # print("Estimated BPM:", global_tempo) #67


        filename_column.append(filename)
        BPM_column.append(global_tempo)

        print(f'{filename} | {global_tempo}')


create_CSV(filename_column, BPM_column, "test_BPM_TempoCNN [deeptemp-k4].csv")






print("deeptemp-k16")
filename_column = []
BPM_column = []
# Loop through each file in the directory
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)

    # Check if it's a file (not a subdirectory)
    if os.path.isfile(file_path):


        # Essentia TempoCNN
        # 3 different models: the k16 ones are more precise, using more resources
        # SEMPRE PRECISO, NON SBAGLIA MAI (PER ORA)


        # PROCESS:
        from essentia.standard import MonoLoader, TempoCNN

        import librosa
        y, sr = librosa.load(file_path)

        audio = MonoLoader(filename=file_path, sampleRate=sr, resampleQuality=4)()


        model3 = TempoCNN(graphFilename="/content/deeptemp-k16-3.pb")
        global_tempo, local_tempo, local_tempo_probabilities = model3(audio)
        # print("Estimated BPM:", global_tempo) #67


        filename_column.append(filename)
        BPM_column.append(global_tempo)

        print(f'{filename} | {global_tempo}')


create_CSV(filename_column, BPM_column, "test_BPM_TempoCNN [deeptemp-k16].csv")

deepsquare-k16
The Winstons - Amen Brother drum loop 138 BPM.wav | 138.0
mi manchi 1 4bars drop, 125bpm.mp3 | 125.0
garage solo drum loop 134 bpm V0.mp3 | 134.0
idea edo suno steelpan V1.1.mp3 | 140.0
african drum fills, 128 bpm..wav | 128.0
techno kick, 135 bpm..wav | 134.0
prova brano fred again stile turn off the lights 134bpm V4.flp.mp3 | 134.0
bulgarian-choir-streets_D_minor.wav | 94.0
garage V5, 140bpm.mp3 | 140.0
mi manchi 2 V4.7.mp3 | 73.0
deeptemp-k4
The Winstons - Amen Brother drum loop 138 BPM.wav | 138.0
mi manchi 1 4bars drop, 125bpm.mp3 | 125.0
garage solo drum loop 134 bpm V0.mp3 | 67.0
idea edo suno steelpan V1.1.mp3 | 140.0
african drum fills, 128 bpm..wav | 128.0
techno kick, 135 bpm..wav | 135.0
prova brano fred again stile turn off the lights 134bpm V4.flp.mp3 | 134.0
bulgarian-choir-streets_D_minor.wav | 97.0
garage V5, 140bpm.mp3 | 140.0
mi manchi 2 V4.7.mp3 | 73.0
deeptemp-k16
The Winstons - Amen Brother drum loop 138 BPM.wav | 138.0
mi manchi 1 4bars drop, 125bp

In [ ]:
import essentia
print(essentia.__version__)

2.1-beta6-dev


# KEY Music feature extraction models/libraries for key test

create folder "/content/3) Loops per prove KEY"


and put inside the files to test

In [ ]:
!pip install essentia
!pip install librosa

In [ ]:
# GENERAL FUNCTIONS FOR ESSENTIA

#import json and assign column to list, returns this list:
def list_classes_from_json(file_path, column_name):
  import json
  # Load the classes from the JSON file
  with open(file_path, 'r') as f:
      data = json.load(f)
  # Extract the classes
  list_classes = data[column_name]
  # OUTPUT:
  return list_classes


#process audio for essentia
def processed_audio_essentia(fileaudiopath):
  from essentia.standard import MonoLoader
  import librosa
  # Load your audio file
  y, sr = librosa.load(fileaudiopath)
  # Process the audio
  audio = MonoLoader(filename=fileaudiopath, sampleRate=sr, resampleQuality=4)()
  # OUTPUT:
  return audio

#creates CSV of column1, column2 ES. filename | result
def create_CSV(column1, column2, filename):
  import csv
  with open(filename, 'w', newline='') as csvfile:
      writer = csv.writer(csvfile)
      for item1, item2 in zip(column1, column2):
        writer.writerow([item1, item2])

In [ ]:
# KEY & SCALE DETECTOR function def:

def key_scale_detector(fileaudiopath):
  # PROCESS:
  import essentia.standard as es

  # Load an audio file
  audio = es.MonoLoader(filename=fileaudiopath)()

  # Detect the key
  key_detector = es.KeyExtractor()
  key, scale, strength = key_detector(audio)

  return f"{key} {scale} key"


# LIMITATIONS:
# is pretty good at determining the key from the fundamental note role:
# pretty smart, as far as my test it can distinguish if a song is D minor instead of G minor considering the fundamental note role,
# without being influenced by harmonic or melodic minor scale alterations.
# but the scale detection in *only major or minor in this implementation*
# -> FUTURE PROJECT: extend the scale detection profiles of essentia to recognize all the scale modes: iodian, lydian, mixolydian ecc...

# IN THEORY, this wont allow for strange music styles like uk house mono-chord songs, but gotta test it, maybe i find a way to make it work







import os

# INPUT
directory_path = '/content/3) Loops per prove KEY'

filename_column = []
key_column = []
# Loop through each file in the directory
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)

    # Check if it's a file (not a subdirectory)
    if os.path.isfile(file_path):


        # KEY & SCALE DETECTOR of input music loop
        # also in this case essentia works much better than librosa, giving key and the scale

        # PROCESS:
        key_scale_input = key_scale_detector(file_path)

        # OUTPUT:
        print(f'{filename} | {key_scale_input}')



        filename_column.append(filename)
        key_column.append(key_scale_input)


create_CSV(filename_column, key_column, "test_key_KeyExtractor.csv")

13) B min - 574491__deadrobotmusic__upright-piano-loop-3-125bpm-b-minor.wav | D major key
10) D maj - 13921__robcro6010__stacatto-rhythm.m4a | D minor key
19) F min - 670030__harrisonlace__synth_loop_replicant_fm_175.wav | Bb minor key
6) Eb maj - 395960__tripjazz__jonova.aiff | Bb major key
4) G min - 357071__mtg__clarinet-g-natural-minor.wav | C minor key
9) Db maj - 505919__tripjazz__hobi-polo.wav | B major key
14) A min - 574602__deadrobotmusic__nylon-guitar-loop-104bpm-a-minor.wav | A minor key
16) F# min - 578699__badoink__storm-rider.wav | F# minor key
17) D min - 621464__onlineforever__wilson-118bpm-d-minor-kalimba.wav | D minor key
8) C maj - 477209__enviromaniac2__tune-in-c-major-strings-and-synth.mp3 | C major key
5) Bb min - 395035__tripjazz__nobel A musical piece created in the "Figure" app. 120 (60) bpm Bb minor..aiff | F major key
2) c min - 30266__unclesigmund__a_cminor_bminor_e_chord_progression_on_guitar_with_echo.wav | Eb major key
7) Db maj - 401019__tripjazz__bimbo

# GENRE feature extraction test:

i modelli restituiscono un tag per momenti:
ho provato con brani di varia durata:
- 8s -> 10 tags
- 286s -> circa 395 tags
- 73 -> circa 99 tags

insomma 1 tag ogni 700/800 ms. i tag comunque sono molto simili, si potrebbe fare una lista da dare al modello (convertendo in tupla), oppure scegliere con metodo di maggioranza e tenere un solo genere.


questo vuol dire che il nparray in output è di dimensione class x durata: esempio 400(generi) x 10(momenti)


GUIDA:
create folder "/content/3) Loops per prove KEY" , and put inside the files to test

In [ ]:
!pip install essentia
!pip install librosa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 107.6 MB/s eta 0:00:00


In [ ]:
# unistall tensorflow if needed
# !pip uninstall tensorflow -y

!pip install essentia-tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.4/291.4 MB 2.6 MB/s eta 0:00:00


In [ ]:
#import json and assign column to list, returns this list:
def list_classes_from_json(file_path, column_name):
  import json
  # Load the classes from the JSON file
  with open(file_path, 'r') as f:
      data = json.load(f)
  # Extract the classes
  list_classes = data[column_name]
  # OUTPUT:
  return list_classes


#process audio for essentia
def processed_audio_essentia(fileaudiopath):
  from essentia.standard import MonoLoader
  import librosa
  # Load your audio file
  y, sr = librosa.load(fileaudiopath)
  # Process the audio
  audio = MonoLoader(filename=fileaudiopath, sampleRate=sr, resampleQuality=4)()
  # OUTPUT:
  return audio

#creates CSV of column1, column2 ES. filename | result
def create_CSV(column1, column2, filename):
  import csv
  with open(filename, 'w', newline='') as csvfile:
      writer = csv.writer(csvfile)
      for item1, item2 in zip(column1, column2):
        writer.writerow([item1, item2])

In [ ]:
# download genre_discogs400 model weights
!wget https://essentia.upf.edu/models/classification-heads/genre_discogs400/genre_discogs400-discogs-effnet-1.pb
!wget https://essentia.upf.edu/models/music-style-classification/discogs-effnet/discogs-effnet-bs64-1.pb
# download MTG-Jamendo genre model weights
!wget https://essentia.upf.edu/models/classification-heads/mtg_jamendo_genre/mtg_jamendo_genre-discogs-effnet-1.pb

--2024-11-01 05:52:32--  https://essentia.upf.edu/models/classification-heads/genre_discogs400/genre_discogs400-discogs-effnet-1.pb
Resolving essentia.upf.edu (essentia.upf.edu)... 84.89.139.43
Connecting to essentia.upf.edu (essentia.upf.edu)|84.89.139.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2057977 (2.0M) [application/octet-stream]
Saving to: ‘genre_discogs400-discogs-effnet-1.pb’

genre_discogs400-di 100%[===================>]   1.96M   516KB/s    in 3.9s    

2024-11-01 05:52:37 (516 KB/s) - ‘genre_discogs400-discogs-effnet-1.pb’ saved [2057977/2057977]

--2024-11-01 05:52:37--  https://essentia.upf.edu/models/music-style-classification/discogs-effnet/discogs-effnet-bs64-1.pb
Resolving essentia.upf.edu (essentia.upf.edu)... 84.89.139.43
Connecting to essentia.upf.edu (essentia.upf.edu)|84.89.139.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18366619 (18M) [application/octet-stream]
Saving to: ‘discogs-effnet-bs64-1

In [ ]:
!wget https://essentia.upf.edu/models/classification-heads/genre_discogs400/genre_discogs400-discogs-effnet-1.json

--2024-11-01 05:53:14--  https://essentia.upf.edu/models/classification-heads/genre_discogs400/genre_discogs400-discogs-effnet-1.json
Resolving essentia.upf.edu (essentia.upf.edu)... 84.89.139.43
Connecting to essentia.upf.edu (essentia.upf.edu)|84.89.139.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14951 (15K) [application/json]
Saving to: ‘genre_discogs400-discogs-effnet-1.json’

genre_discogs400-di 100%[===================>]  14.60K  65.0KB/s    in 0.2s    

2024-11-01 05:53:15 (65.0 KB/s) - ‘genre_discogs400-discogs-effnet-1.json’ saved [14951/14951]



In [ ]:
import os

# INPUT
directory_path = '/content/4) Loops per prove genre'



filename_column = []
genre_column = []
# Loop through each file in the directory
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)

    # Check if it's a file (not a subdirectory)
    if os.path.isfile(file_path):


        # Essentia genre_discogs400 (400 classes)
        # PROCESS:
        import numpy as np
        from essentia.standard import MonoLoader, TensorflowPredictEffnetDiscogs, TensorflowPredict2D
        import librosa


        audio = processed_audio_essentia(file_path)
        genres = list_classes_from_json("/content/genre_discogs400-discogs-effnet-1.json", 'classes')


        embedding_model = TensorflowPredictEffnetDiscogs(graphFilename="/content/discogs-effnet-bs64-1.pb", output="PartitionedCall:1")
        embeddings = embedding_model(audio)

        model = TensorflowPredict2D(graphFilename="/content/genre_discogs400-discogs-effnet-1.pb", input="serving_default_model_Placeholder", output="PartitionedCall:0")
        predictions = model(embeddings)




        time_tags_list = []
        # OUTPUT:
        for i in range(0, len(predictions)):
          time_tags_list.append(genres[np.argmax(predictions[i])])

        from collections import Counter
        time_tags_list_count = Counter(time_tags_list)
        time_tags_list_count = dict(time_tags_list_count)

        genre_dict = time_tags_list_count
        genre_dict_sorted = dict(sorted(genre_dict.items(), key=lambda item: item[1], reverse=True))

        filename_column.append(filename)
        genre_column.append(genre_dict_sorted)

        print(f'{filename} | {genre_dict_sorted}')


create_CSV(filename_column, genre_column, "test_genre_Essentia genre_discogs400.csv")


561390__migfus20__funny-background-music-orchestra-loop.mp3 | {'Classical---Contemporary': 26, 'Classical---Neo-Romantic': 15, 'Stage & Screen---Soundtrack': 8, 'Classical---Modern': 8, 'Stage & Screen---Score': 6, 'Jazz---Easy Listening': 3, 'Classical---Impressionist': 3, 'Brass & Military---Marches': 2, 'Classical---Post-Modern': 2, 'Classical---Romantic': 1, 'Electronic---Dungeon Synth': 1, 'Electronic---Experimental': 1, 'Rock---Black Metal': 1}
remics lollo filo you and me flume V3.mp3 | {'Electronic---Ambient': 93, 'Electronic---Experimental': 77, 'Electronic---Downtempo': 61, 'Electronic---Vaporwave': 51, 'Electronic---House': 30, 'Stage & Screen---Soundtrack': 22, 'Brass & Military---Military': 8, 'Classical---Post-Modern': 8, 'Electronic---Psy-Trance': 8, 'Classical---Neo-Romantic': 5, 'Funk / Soul---Soul': 4, 'Hip Hop---RnB/Swing': 4, 'Electronic---Synth-pop': 4, 'Classical---Contemporary': 4, 'Rock---Pop Rock': 4, 'Rock---AOR': 4, 'Electronic---Industrial': 4, 'Classical---

In [ ]:
!wget https://essentia.upf.edu/models/classification-heads/mtg_jamendo_genre/mtg_jamendo_genre-discogs-effnet-1.json

--2024-11-01 06:06:10--  https://essentia.upf.edu/models/classification-heads/mtg_jamendo_genre/mtg_jamendo_genre-discogs-effnet-1.json
Resolving essentia.upf.edu (essentia.upf.edu)... 84.89.139.43
Connecting to essentia.upf.edu (essentia.upf.edu)|84.89.139.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4267 (4.2K) [application/json]
Saving to: ‘mtg_jamendo_genre-discogs-effnet-1.json’

mtg_jamendo_genre-d 100%[===================>]   4.17K  --.-KB/s    in 0s      

2024-11-01 06:06:11 (45.0 MB/s) - ‘mtg_jamendo_genre-discogs-effnet-1.json’ saved [4267/4267]



In [ ]:
import os

# INPUT
directory_path = '/content/4) Loops per prove genre'



filename_column = []
genre_column = []
# Loop through each file in the directory
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)

    # Check if it's a file (not a subdirectory)
    if os.path.isfile(file_path):


        # Essentia MTG-Jamendo genre (87 classes)

        # PROCESS:
        import numpy as np
        import librosa
        from essentia.standard import MonoLoader, TensorflowPredictEffnetDiscogs, TensorflowPredict2D


        audio = processed_audio_essentia(file_path)
        genres = list_classes_from_json("/content/mtg_jamendo_genre-discogs-effnet-1.json", 'classes')


        embedding_model = TensorflowPredictEffnetDiscogs(graphFilename="/content/discogs-effnet-bs64-1.pb", output="PartitionedCall:1")
        embeddings = embedding_model(audio)

        model = TensorflowPredict2D(graphFilename="mtg_jamendo_genre-discogs-effnet-1.pb")
        predictions = model(embeddings)



        time_tags_list = []
        # OUTPUT:
        for i in range(0, len(predictions)):
          time_tags_list.append(genres[np.argmax(predictions[i])])



        from collections import Counter
        time_tags_list_count = Counter(time_tags_list)
        time_tags_list_count = dict(time_tags_list_count)

        genre_dict = time_tags_list_count
        genre_dict_sorted = dict(sorted(genre_dict.items(), key=lambda item: item[1], reverse=True))

        filename_column.append(filename)
        genre_column.append(genre_dict_sorted)

        print(f'{filename} | {genre_dict_sorted}')


create_CSV(filename_column, genre_column, "test_genre_Essentia MTG-Jamendo genre.csv")













# 60s, 70s, 80s, 90s, acidjazz, alternative, alternativerock, ambient, atmospheric, blues, bluesrock, bossanova, breakbeat,
# celtic, chanson, chillout, choir, classical, classicrock, club, contemporary, country, dance, darkambient, darkwave,
# deephouse, disco, downtempo, drumnbass, dub, dubstep, easylistening, edm, electronic, electronica, electropop, ethno,
# eurodance, experimental, folk, funk, fusion, groove, grunge, hard, hardrock, hiphop, house, idm, improvisation, indie,
# industrial, instrumentalpop, instrumentalrock, jazz, jazzfusion, latin, lounge, medieval, metal, minimal, newage, newwave,
# orchestral, pop, popfolk, poprock, postrock, progressive, psychedelic, punkrock, rap, reggae, rnb, rock, rocknroll,
# singersongwriter, soul, soundtrack, swing, symphonic, synthpop, techno, trance, triphop, world, worldfusion


561390__migfus20__funny-background-music-orchestra-loop.mp3 | {'classical': 74, 'soundtrack': 3}
remics lollo filo you and me flume V3.mp3 | {'electronic': 243, 'ambient': 58, 'classical': 56, 'chillout': 56, 'hiphop': 7, 'pop': 6, 'alternative': 1}
Good morning Kanye.wav | {'hiphop': 10, 'electronic': 3, 'pop': 2}
skyfall remix V5.mp3 | {'electronic': 174, 'ambient': 134, 'classical': 75, 'alternative': 15, 'chillout': 15, 'indie': 12, 'pop': 10, 'rock': 3, 'experimental': 3, 'house': 2, 'hiphop': 1}
The Winstons - Amen Brother drum loop 138 BPM.wav | {'hiphop': 5, 'electronic': 3}
17) D min - 621464__onlineforever__wilson-118bpm-d-minor-kalimba.wav | {'electronic': 25, 'ambient': 5}
garage solo drum loop 134 bpm V0.mp3 | {'electronic': 25, 'hiphop': 15, 'alternative': 1}
Lucio Battisti - Con il nastro rosa remix V3 FINITA PRODUZIONE.mp3 | {'electronic': 231, 'pop': 44, 'chillout': 39, 'jazz': 12, 'hiphop': 8, 'ambient': 4, 'alternative': 4, 'lounge': 2, 'world': 2, 'soundtrack': 1, '

# MOOD feature extraction test

Jamendo mood and theme filtato è buono, l'altro non serve a nulla, 5 classi alla fine



GUIDA:
create folder "/content/5) loops per prove moods" , and put inside the files to test

In [ ]:

# unistall tensorflow if needed
# !pip uninstall tensorflow -y

!pip install essentia-tensorflow

In [ ]:
#import json and assign column to list, returns this list:
def list_classes_from_json(file_path, column_name):
  import json
  # Load the classes from the JSON file
  with open(file_path, 'r') as f:
      data = json.load(f)
  # Extract the classes
  list_classes = data[column_name]
  # OUTPUT:
  return list_classes


#process audio for essentia
def processed_audio_essentia(fileaudiopath):
  from essentia.standard import MonoLoader
  import librosa
  # Load your audio file
  y, sr = librosa.load(fileaudiopath)
  # Process the audio
  audio = MonoLoader(filename=fileaudiopath, sampleRate=sr, resampleQuality=4)()
  # OUTPUT:
  return audio

#creates CSV of column1, column2 ES. filename | result
def create_CSV(column1, column2, filename):
  import csv
  with open(filename, 'w', newline='') as csvfile:
      writer = csv.writer(csvfile)
      for item1, item2 in zip(column1, column2):
        writer.writerow([item1, item2])


In [ ]:
# Essentia MTG-Jamendo mood and theme (56 classes)
# weigths:
!wget https://essentia.upf.edu/models/classification-heads/mtg_jamendo_moodtheme/mtg_jamendo_moodtheme-discogs-effnet-1.pb
#json for classes:
!wget https://essentia.upf.edu/models/classification-heads/mtg_jamendo_moodtheme/mtg_jamendo_moodtheme-discogs-effnet-1.json

#embedding model? always the same right?
!wget https://essentia.upf.edu/models/music-style-classification/discogs-effnet/discogs-effnet-bs64-1.pb

--2024-11-01 06:34:45--  https://essentia.upf.edu/models/classification-heads/mtg_jamendo_moodtheme/mtg_jamendo_moodtheme-discogs-effnet-1.pb
Resolving essentia.upf.edu (essentia.upf.edu)... 84.89.139.43
Connecting to essentia.upf.edu (essentia.upf.edu)|84.89.139.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2739668 (2.6M) [application/octet-stream]
Saving to: ‘mtg_jamendo_moodtheme-discogs-effnet-1.pb’

mtg_jamendo_moodthe 100%[===================>]   2.61M   613KB/s    in 4.7s    

2024-11-01 06:34:51 (566 KB/s) - ‘mtg_jamendo_moodtheme-discogs-effnet-1.pb’ saved [2739668/2739668]

--2024-11-01 06:34:51--  https://essentia.upf.edu/models/classification-heads/mtg_jamendo_moodtheme/mtg_jamendo_moodtheme-discogs-effnet-1.json
Resolving essentia.upf.edu (essentia.upf.edu)... 84.89.139.43
Connecting to essentia.upf.edu (essentia.upf.edu)|84.89.139.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3646 (3.6K) [application/json]
Savi

In [ ]:
# Essentia MTG-Jamendo mood and theme (56 classes)


# INPUT:
# fileaudiopath = input_musicidea_path


#PROCESS:
# function to be used also in the following point
def mood_detector_Essentia_Essentia_MTGJamendo(fileaudiopath):
  from essentia.standard import MonoLoader, TensorflowPredictEffnetDiscogs, TensorflowPredict2D
  import numpy as np


  audio = processed_audio_essentia(fileaudiopath)
  moods = list_classes_from_json("/content/mtg_jamendo_moodtheme-discogs-effnet-1.json", 'classes')


  embedding_model = TensorflowPredictEffnetDiscogs(graphFilename="/content/discogs-effnet-bs64-1.pb", output="PartitionedCall:1")
  embeddings = embedding_model(audio)

  model = TensorflowPredict2D(graphFilename="/content/mtg_jamendo_moodtheme-discogs-effnet-1.pb")
  predictions = model(embeddings)


  time_tags_list = []
  # OUTPUT:
  for i in range(0, len(predictions)):
    time_tags_list.append(moods[np.argmax(predictions[i])])

  from collections import Counter
  time_tags_list_count = Counter(time_tags_list)
  time_tags_list_count = dict(time_tags_list_count)

  return time_tags_list_count


# OUTPUT:
# mood_dict = mood_detector_Essentia_Essentia_MTGJamendo(fileaudiopath)
# print(mood_dict)


# action, adventure, advertising, background, ballad, calm, children, christmas, commercial, cool, corporate, dark, deep,
# documentary, drama, dramatic, dream, emotional, energetic, epic, fast, film, fun, funny, game, groovy, happy, heavy,
# holiday, hopeful, inspiring, love, meditative, melancholic, melodic, motivational, movie, nature, party, positive,
# powerful, relaxing, retro, romantic, sad, sexy, slow, soft, soundscape, space, sport, summer, trailer, travel, upbeat,
# uplifting


#filtered tags list
mood_tags_filtered_list = [
    "calm", "cool", "dream", "emotional", "energetic",
    "fun", "funny", "groovy", "happy", "heavy", "hopeful", "inspiring",
    "love", "meditative", "melancholic", "melodic", "motivational",
    "positive", "powerful", "relaxing", "romantic", "sad", "sexy", "soft",
    "upbeat", "uplifting"]


# mood_dict_filtered = {key: value for key, value in mood_dict.items() if key in mood_tags_filtered_list}
# print(mood_dict_filtered)




In [ ]:
#mood_detector_Essentia_Essentia_MTGJamendo

import os

# INPUT
directory_path = '/content/5) loops per prove moods'


filename_column = []
mood_column = []
# Loop through each file in the directory
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)

    # Check if it's a file (not a subdirectory)
    if os.path.isfile(file_path):

        # OUTPUT RAW:
        mood_dict = mood_detector_Essentia_Essentia_MTGJamendo(file_path)
        mood_dict_sorted = dict(sorted(mood_dict.items(), key=lambda item: item[1], reverse=True))
        print(f'{filename[:6]} | {mood_dict_sorted}')


        # from collections import Counter
        # time_tags_list_count = Counter(time_tags_list)
        # time_tags_list_count = dict(time_tags_list_count)


        filename_column.append(filename)
        mood_column.append(mood_dict_sorted)


create_CSV(filename_column, mood_column, "test_mood_Essentia MTG-Jamendo mood.csv")





755810 | {'space': 45, 'relaxing': 31, 'meditative': 15, 'melodic': 2, 'film': 2, 'dark': 1}
650939 | {'dark': 39, 'film': 30, 'happy': 1}
561390 | {'film': 77}
garage | {'deep': 71, 'energetic': 20, 'love': 4, 'film': 1, 'relaxing': 1, 'melodic': 1, 'soundscape': 1}
Good m | {'love': 13, 'relaxing': 2}
660519 | {'space': 14, 'dark': 4, 'film': 2, 'melodic': 1}
756391 | {'relaxing': 28, 'meditative': 19, 'film': 3, 'soundscape': 1}
The Wi | {'film': 4, 'energetic': 4}
510954 | {'film': 39, 'meditative': 29, 'relaxing': 20, 'soundscape': 1}
garage | {'deep': 16, 'dark': 15, 'slow': 6, 'space': 2, 'energetic': 1, 'love': 1}
652275 | {'energetic': 32}
157008 | {'relaxing': 27, 'meditative': 13, 'film': 12}
383031 | {'space': 14, 'meditative': 11, 'relaxing': 4, 'film': 3}
Martin | {'deep': 19, 'energetic': 18, 'dark': 12, 'melodic': 8, 'space': 3}
160675 | {'meditative': 32, 'film': 13, 'relaxing': 2}
728747 | {'dark': 56, 'meditative': 17, 'film': 14, 'soundscape': 11, 'relaxing': 9, 'sp

In [ ]:
#mood_detector_Essentia_Essentia_MTGJamendo

import os

# INPUT
directory_path = '/content/5) loops per prove moods'


filename_column = []
mood_column = []
# Loop through each file in the directory
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)

    # Check if it's a file (not a subdirectory)
    if os.path.isfile(file_path):

        # OUTPUT RAW:


        mood_tags_filtered_list = [
            "calm", "cool", "dark", "dream", "emotional", "energetic",
            "fun", "funny", "groovy", "happy", "heavy", "hopeful", "inspiring",
            "love", "meditative", "melancholic", "melodic", "motivational",
            "positive", "powerful", "relaxing", "romantic", "sad", "sexy",
            "slow", "soft", "upbeat", "uplifting"]

        mood_dict = mood_detector_Essentia_Essentia_MTGJamendo(file_path)
        mood_dict_filtered = {key: value for key, value in mood_dict.items() if key in mood_tags_filtered_list}
        mood_dict_filtered_sorted = dict(sorted(mood_dict_filtered.items(), key=lambda item: item[1], reverse=True))
        print(f'{filename[:6]} | {mood_dict_filtered_sorted}')



        print(f'{filename[:6]} | {mood_dict_sorted}')


        filename_column.append(filename)
        mood_column.append(mood_dict_filtered_sorted)


create_CSV(filename_column, mood_column, "test_mood_Essentia MTG-Jamendo mood_filtered.csv")





755810 | {'relaxing': 31, 'meditative': 15, 'melodic': 2, 'dark': 1}
755810 | {'deep': 9, 'love': 6, 'melodic': 2, 'documentary': 2, 'happy': 1, 'energetic': 1}
650939 | {'dark': 39, 'happy': 1}
650939 | {'deep': 9, 'love': 6, 'melodic': 2, 'documentary': 2, 'happy': 1, 'energetic': 1}
561390 | {}
561390 | {'deep': 9, 'love': 6, 'melodic': 2, 'documentary': 2, 'happy': 1, 'energetic': 1}
garage | {'energetic': 20, 'love': 4, 'relaxing': 1, 'melodic': 1}
garage | {'deep': 9, 'love': 6, 'melodic': 2, 'documentary': 2, 'happy': 1, 'energetic': 1}
Good m | {'love': 13, 'relaxing': 2}
Good m | {'deep': 9, 'love': 6, 'melodic': 2, 'documentary': 2, 'happy': 1, 'energetic': 1}
660519 | {'dark': 4, 'melodic': 1}
660519 | {'deep': 9, 'love': 6, 'melodic': 2, 'documentary': 2, 'happy': 1, 'energetic': 1}
756391 | {'relaxing': 28, 'meditative': 19}
756391 | {'deep': 9, 'love': 6, 'melodic': 2, 'documentary': 2, 'happy': 1, 'energetic': 1}
The Wi | {'energetic': 4}
The Wi | {'deep': 9, 'love': 6, 

In [ ]:
#input loop to test with all the files

import os

# INPUT
directory_path = '/content/loops per prove moods'

# Loop through each file in the directory
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)

    # Check if it's a file (not a subdirectory)
    if os.path.isfile(file_path):

        # OUTPUT FILTRATO:
        #lista filtrata
        mood_tags_filtered_list = [
            "calm", "cool", "dark", "dream", "emotional", "energetic",
            "fun", "funny", "groovy", "happy", "heavy", "hopeful", "inspiring",
            "love", "meditative", "melancholic", "melodic", "motivational",
            "positive", "powerful", "relaxing", "romantic", "sad", "sexy",
            "slow", "soft", "upbeat", "uplifting"]

        mood_dict = mood_detector_Essentia_Essentia_MTGJamendo(file_path)
        mood_dict_filtered = {key: value for key, value in mood_dict.items() if key in mood_tags_filtered_list}
        mood_dict_filtered_sorted = dict(sorted(mood_dict_filtered.items(), key=lambda item: item[1], reverse=True))
        print(f'{filename[:6]} | {mood_dict_filtered_sorted}')



756391 | {'relaxing': 28, 'meditative': 19}
760263 | {'dark': 21, 'energetic': 10}
Martin | {'energetic': 18, 'dark': 12, 'melodic': 8}
510954 | {'meditative': 29, 'relaxing': 20}
The Wi | {'energetic': 4}
Good m | {'love': 13, 'relaxing': 2}
garage | {'energetic': 20, 'love': 4, 'relaxing': 1, 'melodic': 1}
561390 | {}
157008 | {'relaxing': 27, 'meditative': 13}
654964 | {'dark': 23, 'love': 2, 'relaxing': 1, 'meditative': 1}
650939 | {'dark': 39, 'happy': 1}
410574 | {'love': 6, 'melodic': 2, 'happy': 1, 'energetic': 1}
755810 | {'relaxing': 31, 'meditative': 15, 'melodic': 2, 'dark': 1}
511437 | {'meditative': 24, 'relaxing': 14}
760571 | {'meditative': 48, 'relaxing': 43, 'dark': 2}
728747 | {'dark': 56, 'meditative': 17, 'relaxing': 9, 'love': 2, 'inspiring': 1, 'melodic': 1, 'motivational': 1}
752145 | {'slow': 1, 'relaxing': 1}
760138 | {'relaxing': 145, 'dark': 28, 'meditative': 24, 'slow': 3}
652275 | {'energetic': 32}
garage | {'dark': 15, 'slow': 6, 'energetic': 1, 'love': 1

In [ ]:
#Moods MIREX (29 classes in 5 mood clusters)

#vggish
# weigths:
!wget https://essentia.upf.edu/models/classification-heads/moods_mirex/moods_mirex-audioset-vggish-1.pb
#json for classes:
!wget https://essentia.upf.edu/models/classification-heads/moods_mirex/moods_mirex-audioset-vggish-1.json
# embedding:
!wget https://essentia.upf.edu/models/feature-extractors/vggish/audioset-vggish-3.pb

#musicnn
# weigths:
!wget https://essentia.upf.edu/models/classification-heads/moods_mirex/moods_mirex-msd-musicnn-1.pb
#json for classes:
!wget https://essentia.upf.edu/models/classification-heads/moods_mirex/moods_mirex-msd-musicnn-1.json
# embedding:
!wget https://essentia.upf.edu/models/autotagging/msd/msd-musicnn-1.pb

--2024-11-01 20:06:48--  https://essentia.upf.edu/models/classification-heads/moods_mirex/moods_mirex-audioset-vggish-1.pb
Resolving essentia.upf.edu (essentia.upf.edu)... 84.89.139.43
Connecting to essentia.upf.edu (essentia.upf.edu)|84.89.139.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63015 (62K) [application/octet-stream]
Saving to: ‘moods_mirex-audioset-vggish-1.pb’

moods_mirex-audiose 100%[===================>]  61.54K   123KB/s    in 0.5s    

2024-11-01 20:06:50 (123 KB/s) - ‘moods_mirex-audioset-vggish-1.pb’ saved [63015/63015]

--2024-11-01 20:06:50--  https://essentia.upf.edu/models/classification-heads/moods_mirex/moods_mirex-audioset-vggish-1.json
Resolving essentia.upf.edu (essentia.upf.edu)... 84.89.139.43
Connecting to essentia.upf.edu (essentia.upf.edu)|84.89.139.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2283 (2.2K) [application/json]
Saving to: ‘moods_mirex-audioset-vggish-1.json’

moods_mirex-audios

In [ ]:
# Moods MIREX (29 classes in 5 mood clusters)

import os

# INPUT
directory_path = '/content/5) loops per prove moods'


print("modello musicnn:")


filename_column = []
mood_column = []
# Loop through each file in the directory
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)

    # Check if it's a file (not a subdirectory)
    if os.path.isfile(file_path):







        import numpy as np
        import librosa
        #from essentia.standard import MonoLoader, TensorflowPredictEffnetDiscogs, TensorflowPredict2D

        # INPUT:
        fileaudiopath = "/content/5) loops per prove moods/650939__beetlemuse__numskull.wav"



        #PROCESS:
        audio = processed_audio_essentia(file_path)
        moods = list_classes_from_json("/content/moods_mirex-msd-musicnn-1.json", 'classes')

        # moods_mirex-msd-musicnn
        from essentia.standard import MonoLoader, TensorflowPredictMusiCNN, TensorflowPredict2D

        embedding_model = TensorflowPredictMusiCNN(graphFilename="/content/msd-musicnn-1.pb", output="model/dense/BiasAdd")
        embeddings = embedding_model(audio)

        model = TensorflowPredict2D(graphFilename="/content/moods_mirex-msd-musicnn-1.pb", input="serving_default_model_Placeholder", output="PartitionedCall")
        predictions = model(embeddings)




        time_tags_list = []
        # OUTPUT:
        for i in range(0, len(predictions)):
          time_tags_list.append(moods[np.argmax(predictions[i])])

        from collections import Counter
        time_tags_list_count = Counter(time_tags_list)
        time_tags_list_count = dict(time_tags_list_count)


        mood_dict = time_tags_list_count
        mood_dict_sorted = dict(sorted(mood_dict.items(), key=lambda item: item[1], reverse=True))
        print(f'{filename[:6]} | {mood_dict_sorted}')


        filename_column.append(filename)
        mood_column.append(mood_dict_sorted)


create_CSV(filename_column, mood_column, "test_mood_Essentia Moods MIREX musicnn.csv")





# 1. passionate, rousing, confident, boisterous, rowdy
# 2. rollicking, cheerful, fun, sweet, amiable/good natured
# 3. literate, poignant, wistful, bittersweet, autumnal, brooding
# 4. humorous, silly, campy, quirky, whimsical, witty, wry
# 5. aggressive, fiery, tense/anxious, intense, volatile, visceral

modello musicnn:
24_sa2 | {'aggressive, fiery, tense/anxious, intense, volatile, visceral': 35, 'passionate, rousing, confident, boisterous, rowdy': 5, 'literate, poignant, wistful, bittersweet, autumnal, brooding': 2}
modello musicnn:
160675 | {'literate, poignant, wistful, bittersweet, autumnal, brooding': 31}
modello musicnn:
garage | {'passionate, rousing, confident, boisterous, rowdy': 49, 'literate, poignant, wistful, bittersweet, autumnal, brooding': 11, 'aggressive, fiery, tense/anxious, intense, volatile, visceral': 4, 'rollicking, cheerful, fun, sweet, amiable/good natured': 2}
modello musicnn:
660519 | {'literate, poignant, wistful, bittersweet, autumnal, brooding': 12, 'aggressive, fiery, tense/anxious, intense, volatile, visceral': 1}
modello musicnn:
561390 | {'humorous, silly, campy, quirky, whimsical, witty, wry': 24, 'rollicking, cheerful, fun, sweet, amiable/good natured': 17, 'literate, poignant, wistful, bittersweet, autumnal, brooding': 10}
modello musicnn:
654964 

In [ ]:
# Moods MIREX (29 classes in 5 mood clusters)

import os

# INPUT
directory_path = '/content/5) loops per prove moods'


print("modello vggish:")


filename_column = []
mood_column = []
# Loop through each file in the directory
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)

    # Check if it's a file (not a subdirectory)
    if os.path.isfile(file_path):







        import numpy as np
        import librosa
        #from essentia.standard import MonoLoader, TensorflowPredictEffnetDiscogs, TensorflowPredict2D


        #PROCESS:
        audio = processed_audio_essentia(file_path)
        moods = list_classes_from_json("/content/moods_mirex-audioset-vggish-1.json", 'classes')

        # moods_mirex-audioset-vggish
        from essentia.standard import MonoLoader, TensorflowPredictVGGish, TensorflowPredict2D

        embedding_model = TensorflowPredictVGGish(graphFilename="/content/audioset-vggish-3.pb", output="model/vggish/embeddings")
        embeddings = embedding_model(audio)

        model = TensorflowPredict2D(graphFilename="/content/moods_mirex-audioset-vggish-1.pb", input="serving_default_model_Placeholder", output="PartitionedCall")
        predictions = model(embeddings)



        time_tags_list = []
        # OUTPUT:
        for i in range(0, len(predictions)):
          time_tags_list.append(moods[np.argmax(predictions[i])])

        from collections import Counter
        time_tags_list_count = Counter(time_tags_list)
        time_tags_list_count = dict(time_tags_list_count)


        mood_dict = time_tags_list_count
        mood_dict_sorted = dict(sorted(mood_dict.items(), key=lambda item: item[1], reverse=True))
        print(f'{filename[:6]} | {mood_dict_sorted}')


        filename_column.append(filename)
        mood_column.append(mood_dict_sorted)


create_CSV(filename_column, mood_column, "test_mood_Essentia Moods MIREX vggish.csv")





# 1. passionate, rousing, confident, boisterous, rowdy
# 2. rollicking, cheerful, fun, sweet, amiable/good natured
# 3. literate, poignant, wistful, bittersweet, autumnal, brooding
# 4. humorous, silly, campy, quirky, whimsical, witty, wry
# 5. aggressive, fiery, tense/anxious, intense, volatile, visceral

modello vggish:
24_sa2 | {'passionate, rousing, confident, boisterous, rowdy': 45, 'aggressive, fiery, tense/anxious, intense, volatile, visceral': 13, 'rollicking, cheerful, fun, sweet, amiable/good natured': 8, 'humorous, silly, campy, quirky, whimsical, witty, wry': 2, 'literate, poignant, wistful, bittersweet, autumnal, brooding': 1}
160675 | {'literate, poignant, wistful, bittersweet, autumnal, brooding': 52}
garage | {'rollicking, cheerful, fun, sweet, amiable/good natured': 70, 'passionate, rousing, confident, boisterous, rowdy': 24, 'aggressive, fiery, tense/anxious, intense, volatile, visceral': 6, 'humorous, silly, campy, quirky, whimsical, witty, wry': 5, 'literate, poignant, wistful, bittersweet, autumnal, brooding': 2}
660519 | {'aggressive, fiery, tense/anxious, intense, volatile, visceral': 12, 'literate, poignant, wistful, bittersweet, autumnal, brooding': 7, 'passionate, rousing, confident, boisterous, rowdy': 4}
561390 | {'literate, poignant, wistful, bittersweet, aut